# Extract fNIRS features
Extract various features from the fNIRS data. Here we extract features using the 'HbT' (total hemoglobin) signal.

Input: Processed samples in folder `REFED-dataset/data`

Output: fNIRS (HbT) features to folder *`'path_features'`*

In [1]:
import os
import numpy as np
from scipy.io import loadmat, savemat
from scipy import signal
from DE_PSD import DE_PSD

### Feature extraction parameters

In [2]:
path_data = './REFED-dataset/data'
path_feature = './REFED-dataset/features'

# STFT parameters
stft_para = {
    'stftn' : 400,      # number of points in STFT
    'fStart': [0.01],   # start frequency of each band
    'fEnd'  : [0.2],    # end frequency of each band
    'fs'    : 40,       # sampling frequency
    'window': 1,        # window length in seconds
}

sub_list = os.listdir(path_data)

if not os.path.exists(path_feature):
    os.makedirs(path_feature)

### Feature extraction function

In [3]:
def feature_fNIRS_sub(path_data, stft_para, save_path=None):
    '''
    Extract fNIRS features for one subject
    Inputs:
        path_data: path to the subject data folder
        stft_para: parameters for STFT and feature extraction
        save_path: path to save the extracted features (if None, do not save)
    '''
    # data_fNIRS: n_channels * n_times
    fNIRS_data = loadmat(os.path.join(path_data, 'fNIRS_videos.mat'))
    fNIRS_feature = {}
    
    for vi in fNIRS_data.keys():
        if 'video_' in vi:
            # resample to 40 Hz, original 1000/21 Hz
            fNIRS_vi = signal.resample_poly(fNIRS_data[vi], up=21, down=25, axis=-1)
            assert (fNIRS_vi.shape[-1] % stft_para['fs'] == 0) or (fNIRS_vi.shape[-1] % stft_para['fs'] == 1), \
                'fNIRS length error: %d \% %d = %d' % (fNIRS_vi.shape[-1], stft_para['fs'], fNIRS_vi.shape[-1] % stft_para['fs'])
            if fNIRS_vi.shape[-1] % stft_para['fs'] != 0:
                fNIRS_vi = fNIRS_vi[:, :, :-1]
            fNIRS_vi = fNIRS_vi.reshape(fNIRS_vi.shape[0], fNIRS_vi.shape[1], -1, stft_para['window']*stft_para['fs']).transpose([2,0,1,3])
            
            fNIRS_vi = fNIRS_vi[:, 2] # Extract features using HbT
            print('  [info] Video %s: %s' % (vi, fNIRS_vi.shape), end=' ')
            
            fNIRS_feature_vi = []
            for d in fNIRS_vi:
                # Calculate the statistical characteristics of f NIRS
                mean_nirs = np.mean(d, axis=-1, keepdims=True)
                min_nirs = np.min(d, axis=-1, keepdims=True)
                max_nirs = np.max(d, axis=-1, keepdims=True)
                std_nirs = np.std(d, axis=-1, keepdims=True)
                slope_nirs = (max_nirs - min_nirs) / (d.shape[-1] - 1)
                psd, de = DE_PSD(d, stft_para)
                # Concatenate mean, psd, and de according to the last dimension
                fNIRS_feature_vi.append(np.concatenate([mean_nirs, min_nirs, max_nirs, std_nirs, slope_nirs, de], -1))
            fNIRS_feature_vi = np.array(fNIRS_feature_vi)
            print('-> Feature: %s' % (fNIRS_feature_vi.shape,))
            
            fNIRS_feature[vi] = fNIRS_feature_vi
            
    if save_path is not None:
        feature_mat_path = os.path.join(save_path, 'fNIRS_videos_feature.mat')
        savemat(feature_mat_path, fNIRS_feature)
        print('[info] Save feature to: %s' % (feature_mat_path))

### Perform feature extraction

In [4]:
for si in sub_list: # for each subject
    print('Processing fNIRS data of subject ID.[%s]...' % si)
    path_data_si = os.path.join(path_data, si)
    path_feature_si = os.path.join(path_feature, si)
    if not os.path.exists(path_feature_si):
        os.makedirs(path_feature_si)
    
    feature_fNIRS_sub(path_data_si, stft_para, save_path=path_feature_si)

Processing fNIRS data of subject ID.[1]...
  [info] Video video_1: (134, 51, 40) -> Feature: (134, 51, 6)
  [info] Video video_2: (137, 51, 40) -> Feature: (137, 51, 6)
  [info] Video video_3: (90, 51, 40) -> Feature: (90, 51, 6)
  [info] Video video_4: (79, 51, 40) -> Feature: (79, 51, 6)
  [info] Video video_5: (135, 51, 40) -> Feature: (135, 51, 6)
  [info] Video video_6: (93, 51, 40) -> Feature: (93, 51, 6)
  [info] Video video_7: (122, 51, 40) -> Feature: (122, 51, 6)
  [info] Video video_8: (107, 51, 40) -> Feature: (107, 51, 6)
  [info] Video video_9: (70, 51, 40) -> Feature: (70, 51, 6)
  [info] Video video_10: (60, 51, 40) -> Feature: (60, 51, 6)
  [info] Video video_11: (61, 51, 40) -> Feature: (61, 51, 6)
  [info] Video video_12: (63, 51, 40) -> Feature: (63, 51, 6)
  [info] Video video_13: (111, 51, 40) -> Feature: (111, 51, 6)
  [info] Video video_14: (103, 51, 40) -> Feature: (103, 51, 6)
  [info] Video video_15: (170, 51, 40) -> Feature: (170, 51, 6)
[info] Save feature 